In [1]:
from src.utils.quantization import export_and_quantize_dinov2, load_input, load_model, export_dinov2, load_input_fixed
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import matplotlib.pyplot as plt
from demo import VisualizationDemo
from detectron2.data.detection_utils import read_image
from copy import copy
import torch
import torchvision.transforms as T

In [2]:
image_path = "artifacts/idea_raw.jpg"
# image_path = "artifacts/white.jpg"
img, example_kwargs = load_input_fixed(image_path=image_path)

In [ ]:
model = load_model()

In [ ]:
exported_model = export_dinov2(model, example_kwargs)

In [5]:
image_path = "artifacts/ams.jpg"
img, example_kwargs = load_input_fixed(image_path=image_path)

In [ ]:
with torch.no_grad():
    out = model(**example_kwargs)
    del model

In [ ]:
with torch.no_grad():
    out_exported = exported_model.module()(**example_kwargs)
    del exported_model

In [8]:
def filter_predictions_with_confidence(predictions, confidence_threshold=0.5):
    if "instances" in predictions:
        preds = predictions["instances"]
        keep_idxs = preds.scores > confidence_threshold
        predictions = copy(predictions) # don't modify the original
        predictions["instances"] = preds[keep_idxs]
    return predictions

In [9]:
out = filter_predictions_with_confidence(out)

In [10]:
out_exported = filter_predictions_with_confidence(out_exported)

In [ ]:

v = Visualizer(img, MetadataCatalog.get("coco_2017_val"))
pred = filter_predictions_with_confidence(out[0], confidence_threshold=0.5)
v = v.draw_instance_predictions(pred["instances"].to("cpu"))

# Display the results
plt.figure(figsize=(14, 10))
plt.imshow(v.get_image()[:, :, ::-1])
plt.axis('off')
plt.show()

In [ ]:

v = Visualizer(img, MetadataCatalog.get("coco_2017_val"))
pred = filter_predictions_with_confidence(out_exported[0], confidence_threshold=0.5)
v = v.draw_instance_predictions(pred["instances"].to("cpu"))

# Display the results
plt.figure(figsize=(14, 10))
plt.imshow(v.get_image()[:, :, ::-1])
plt.axis('off')
plt.show()